# Get Previous Trends in Twitter
using getdaytrends.com

In [ ]:
import re
import pandas as pd
import urllib3
import certifi
import datetime
from bs4 import BeautifulSoup

In [ ]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [ ]:
TRENDS_PATH = "../data/trends/11-06-2020.csv"
HASHTAG_LABEL_PATH = "../data/trends/hashtag_label.xlsx"

In [ ]:
hours = range(0, 23)

In [ ]:
def parse(response, date, hour):
    trends = []
    soup = BeautifulSoup(response.data, "lxml")

    tr_list = soup.find_all("tr")

    for tr in tr_list:
        try:
            trend_name = tr.find("td").find('a').text
            desc = tr.find("td").span.text
            trends.append({"name": trend_name, "description": desc, "date": date, "hour": hour})
        except:
            pass
    return trends

In [ ]:
trends = []
for min_day in range(1, 30):
    print(min_day)
    for hour in hours:
        today = datetime.datetime.today()
        date = today - datetime.timedelta(min_day)
        date = date.strftime("%Y-%m-%d")
        response = http.request('GET', url = "https://getdaytrends.com/indonesia/{}/{}/".format(date, hour))
        trends.extend(parse(response, date, hour))

In [ ]:
d_trends = pd.DataFrame(trends, columns=["name", "description", "date", "hour"])

In [ ]:
d_trends.shape

In [ ]:
d_trends.head()

In [ ]:
d_trends.to_csv(TRENDS_PATH, index=False)

### create doc for label

In [ ]:
list_trends = d_trends.name.value_counts().index

In [ ]:
hashtag_trends = []
for trend in list_trends:
    try:
        t = re.match("#.*", trend).group()
        hashtag_trends.append(t)
    except:
        pass

In [ ]:
d_hashtag_only = pd.DataFrame(data = {"trend": hashtag_trends})

In [ ]:
d_hashtag_only["category"] = 0

In [ ]:
d_hashtag_only.to_excel(HASHTAG_LABEL_PATH, index=False)